In [1]:
import statsmodels.api as sm

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray.plot as xplt
import cftime

In [198]:
cpmdata = xr.open_mfdataset("../../../data/2.2km/rcp85/01/pr/*.nc").rename({"pr": "target_pr"})
gcmdata = xr.open_mfdataset("../../../data/60km/rcp85/01/*/day/*.nc")
regridded_gcmdata = xr.open_mfdataset('../../../derived_data/60km/rcp85/01/*/day/*.nc')

In [199]:
cp_model_rotated_pole = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
platecarree = ccrs.PlateCarree()

In [200]:
cpmdata = cpmdata.loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))].isel(grid_latitude=100, grid_longitude=100)

In [201]:
regridded_gcmdata = regridded_gcmdata.loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))].isel(grid_latitude=100, grid_longitude=100)

In [202]:
cpmdata = cpmdata.reset_coords()[['target_pr']]
cpmdata

<xarray.Dataset>
Dimensions:    (time: 720)
Coordinates:
  * time       (time) object 1980-12-01 12:00:00 ... 1982-11-30 12:00:00
Data variables:
    target_pr  (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
Attributes: (12/15)
    collection:      land-cpm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2019-07-30T06:13:12
    domain:          uk
    frequency:       day
    institution:     Met Office Hadley Centre (MOHC), FitzRoy Road, Exeter, D...
    ...              ...
    resolution:      2.2km
    scenario:        rcp85
    source:          UKCP18 realisation from a set of 12 convection-permittin...
    title:           UKCP18 land projections - 2.2km convection-permitting cl...
    version:         v20190731
    Conventions:     CF-1.5

In [203]:
regridded_gcmdata = regridded_gcmdata.reset_coords()[['pr', 'psl']]
regridded_gcmdata

<xarray.Dataset>
Dimensions:  (time: 720)
Coordinates:
  * time     (time) object 1980-12-01 12:00:00 ... 1982-11-30 12:00:00
Data variables:
    pr       (time) float32 dask.array<chunksize=(720,), meta=np.ndarray>
    psl      (time) float32 dask.array<chunksize=(720,), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [212]:
merged_data = xr.merge([regridded_gcmdata, cpmdata])
training_data = merged_data.sel({"time": slice("1980-12-01", "1981-11-30")})
test_data = merged_data.sel({"time": slice("1981-12-01", "1982-11-30")})

training_data

<xarray.Dataset>
Dimensions:    (time: 360)
Coordinates:
  * time       (time) object 1980-12-01 12:00:00 ... 1981-11-30 12:00:00
Data variables:
    pr         (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
    psl        (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
    target_pr  (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [213]:
test_data

<xarray.Dataset>
Dimensions:    (time: 360)
Coordinates:
  * time       (time) object 1981-12-01 12:00:00 ... 1982-11-30 12:00:00
Data variables:
    pr         (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
    psl        (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
    target_pr  (time) float32 dask.array<chunksize=(360,), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [214]:
X = training_data[['pr', 'psl']].to_pandas()
X = sm.add_constant(X, prepend=False)
y = training_data[['target_pr']].to_pandas()
y['wetday'] = y['target_pr'] > 0.1

X_test = test_data[['pr', 'psl']].to_pandas()
X_test = sm.add_constant(X_test, prepend=False)
y_test = test_data[['target_pr']].to_pandas()
y_test['wetday'] = y_test['target_pr'] > 0.1

In [215]:
X

,pr,psl,const
time,,,
1980-12-01 12:00:00,1.899322,1007.503662,1.0
1980-12-02 12:00:00,2.545550,1015.760193,1.0
1980-12-03 12:00:00,0.450264,1023.282593,1.0
1980-12-04 12:00:00,5.709184,1021.054260,1.0
1980-12-05 12:00:00,4.438090,1014.163818,1.0
...,...,...,...
1981-11-26 12:00:00,0.445454,1002.772156,1.0
1981-11-27 12:00:00,4.356998,1003.470337,1.0
1981-11-28 12:00:00,3.928339,1011.265259,1.0


In [216]:
X_test

,pr,psl,const
time,,,
1981-12-01 12:00:00,0.117201,1019.596375,1.0
1981-12-02 12:00:00,0.967265,1015.763855,1.0
1981-12-03 12:00:00,0.245299,1023.148743,1.0
1981-12-04 12:00:00,7.697859,1022.303406,1.0
1981-12-05 12:00:00,7.504051,1012.779724,1.0
...,...,...,...
1982-11-26 12:00:00,1.885039,1018.941833,1.0
1982-11-27 12:00:00,0.666726,1022.566895,1.0
1982-11-28 12:00:00,3.889669,1023.141296,1.0


In [185]:
y

,target_pr,wetday
time,,
1980-12-01 12:00:00,0.591544,True
1980-12-02 12:00:00,1.416168,True
1980-12-03 12:00:00,3.948448,True
1980-12-04 12:00:00,5.279624,True
1980-12-05 12:00:00,6.765685,True
...,...,...
1981-11-26 12:00:00,0.002237,False
1981-11-27 12:00:00,0.067683,False
1981-11-28 12:00:00,0.826572,True


In [179]:
glm_binom = sm.GLM(y.wetday, X, family=sm.families.Binomial(sm.families.links.logit()))
glm_binom_results = glm_binom.fit()
print(glm_binom_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 wetday   No. Observations:                  360
Model:                            GLM   Df Residuals:                      357
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -152.61
Date:                Mon, 19 Jul 2021   Deviance:                       305.22
Time:                        14:09:21   Pearson chi2:                     464.
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pr             0.8854      0.178      4.988      0.0

In [181]:
glm_gamma = sm.GLM(y[y.wetday].target_pr, X[y.wetday], family=sm.families.Gamma(sm.families.links.log()))
glm_results = glm_gamma.fit()
print(glm_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              target_pr   No. Observations:                  218
Model:                            GLM   Df Residuals:                      215
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                          1.6070
Method:                          IRLS   Log-Likelihood:                -539.60
Date:                Mon, 19 Jul 2021   Deviance:                       260.85
Time:                        14:10:03   Pearson chi2:                     346.
No. Iterations:                    29                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pr             0.0962      0.013      7.350      0.0

In [226]:
glm_binom_results.predict(X_test)

AssertionError: 

In [222]:
y_test

,target_pr,wetday
time,,
1981-12-01 12:00:00,0.355140,True
1981-12-02 12:00:00,0.004497,False
1981-12-03 12:00:00,0.841326,True
1981-12-04 12:00:00,4.947082,True
1981-12-05 12:00:00,12.717894,True
...,...,...
1982-11-26 12:00:00,0.097777,False
1982-11-27 12:00:00,0.138761,True
1982-11-28 12:00:00,0.951559,True


In [111]:
data = np.random.rand(4, 3)
locs = ["IA", "IL", "IN"]
times = pd.date_range("2000-01-01", periods=4)

da = xr.DataArray(
    data,
    coords={
        "time": times,
        "space": locs,
        "const": 42,
        "ranking": (("time", "space"), np.arange(12).reshape(4, 3)),
    },
    dims=["time", "space"],
)

da

<xarray.DataArray (time: 4, space: 3)>
array([[0.38202946, 0.8163229 , 0.35055273],
       [0.20195927, 0.41667519, 0.13855852],
       [0.26462852, 0.5528702 , 0.76470672],
       [0.98113445, 0.1845844 , 0.9131255 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) int64 0 1 2 3 4 5 6 7 8 9 10 11

In [113]:
da.sel(ranking=3)

ValueError: dimensions or multi-index levels ['ranking'] do not exist

In [228]:
da.time

<xarray.DataArray 'time' (time: 4)>
array(['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000',
       '2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
    const    int64 42

In [229]:
data = np.random.rand(4, 3)
locs = ["IA", "IL", "IN"]
times = pd.date_range("2000-01-03", periods=4)

da2 = xr.DataArray(
    data,
    coords={
        "time": times,
        "space": locs,
        "const": 42,
        "ranking": (("time", "space"), np.arange(12).reshape(4, 3)),
    },
    dims=["time", "space"],
)

da2

<xarray.DataArray (time: 4, space: 3)>
array([[0.3032869 , 0.95576475, 0.7872357 ],
       [0.04946353, 0.73932728, 0.04675097],
       [0.17472428, 0.26062227, 0.84704447],
       [0.67943457, 0.86266098, 0.28776212]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-03 2000-01-04 2000-01-05 2000-01-06
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) int64 0 1 2 3 4 5 6 7 8 9 10 11

In [230]:
da

<xarray.DataArray (time: 4, space: 3)>
array([[0.38202946, 0.8163229 , 0.35055273],
       [0.20195927, 0.41667519, 0.13855852],
       [0.26462852, 0.5528702 , 0.76470672],
       [0.98113445, 0.1845844 , 0.9131255 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) int64 0 1 2 3 4 5 6 7 8 9 10 11

In [231]:
da2

<xarray.DataArray (time: 4, space: 3)>
array([[0.3032869 , 0.95576475, 0.7872357 ],
       [0.04946353, 0.73932728, 0.04675097],
       [0.17472428, 0.26062227, 0.84704447],
       [0.67943457, 0.86266098, 0.28776212]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-03 2000-01-04 2000-01-05 2000-01-06
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) int64 0 1 2 3 4 5 6 7 8 9 10 11

In [233]:
da['value'] = xr.full_like(da, fill_value=1)

In [242]:
da.interp_like(da2)

<xarray.DataArray (time: 4, space: 3)>
array([[0.26462852, 0.5528702 , 0.76470672],
       [0.98113445, 0.1845844 , 0.9131255 ],
       [       nan,        nan,        nan],
       [       nan,        nan,        nan]])
Coordinates:
  * space    (space) object 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) float64 6.0 7.0 8.0 9.0 10.0 ... nan nan nan nan nan
    value    (time, space) float64 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan nan
  * time     (time) datetime64[ns] 2000-01-03 2000-01-04 2000-01-05 2000-01-06

In [244]:
da.where(da.time == da2.time)

<xarray.DataArray (time: 2, space: 3)>
array([[0.26462852, 0.5528702 , 0.76470672],
       [0.98113445, 0.1845844 , 0.9131255 ]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int64 42
    ranking  (time, space) int64 6 7 8 9 10 11
    value    (time, space) float64 1.0 1.0 1.0 1.0 1.0 1.0